# Regular Genomics Project **RNA Localisation**


---

**Problem definition**:

TODO


### **1.** First steps with data

Firstly, we import several necessary packages and load in our data:

In [48]:
import pandas as pd
import numpy as np
import itertools
import os
import tensorflow as tf
import sys
from keras.utils import to_categorical
from collections import OrderedDict
from sklearn.model_selection import StratifiedKFold, KFold

In [50]:
# Initializing test set
np.random.seed(3)
data_org = pd.read_csv('final_data.csv')
test_data = data_org.sample(frac=0.1)
train_data = data_org.drop(test_data.index)

data_org

,ERM,KDEL,LMA,MITO,NES,NIK,NLS,NUCP,OMM,gene_id,gene_biotype,seq,struct,m6A_5UTR,m6A_CDS,m6A_3UTR
0,57.045409,35.456782,22.008215,12.355106,22.789983,24.241731,16.970436,29.348389,54.916891,ENSG00000000003,protein_coding,ACCTTGTATTAGGTATTTATTTCCACAAAAGTTTGATGCTTACAAC...,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",0,0,1
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.914814,0.244517,ENSG00000000005,protein_coding,TGTGCACAGAAGTTATATACATATATGGGTATATCTATGTAACAAA...,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",0,0,0
2,17.449430,34.151539,29.338431,22.237585,24.682767,43.612551,38.683963,35.678476,25.348560,ENSG00000000419,protein_coding,TACTTTATGCAAAAAAAAATATACATTTATTTATAGGTCTCAATAC...,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",0,0,0
3,3.830180,2.576734,5.737850,0.761343,2.786808,2.784356,3.382682,2.463676,2.819269,ENSG00000000457,protein_coding,TGACTTTCAAACCATTTTAATATTTCAAATATTCCAGAACAATCCC...,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",0,9,5
4,13.132915,8.782925,10.061390,3.012459,8.821250,6.721117,10.827253,8.005113,6.849962,ENSG00000000460,protein_coding,AACCCGCTCGGGTCCCCTTCCACACTGTGGAAGCTTTGTTCTTTCG...,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13805,0.000000,0.000000,0.506809,0.075893,0.050764,0.000000,0.056586,0.000000,0.000000,ENSG00000281883,protein_coding,GGGAAGAAAGGAGCCTGACTCTTATGATGGAATAACCACAAATCAG...,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",0,2,1
13806,0.105452,0.087130,0.171187,0.016101,0.079057,0.669947,0.171672,0.000000,0.254546,ENSG00000282034,protein_coding,GTGTCGGACGGCATGACAGGCAGCAATCCTGTGTCCCCTGCCTCAT...,"[0.37599998712539673, 0.0, 0.07500000298023224...",0,66,4
13807,0.037093,0.444844,0.425368,0.254467,0.323794,1.803249,1.435483,0.249590,0.483645,ENSG00000282827,protein_coding,ATGGCGGGGACCTCCGCGCCAGGCAGCAAGAGGCGGAGCGAGCCCC...,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",0,0,0
13808,0.000000,0.519421,2.119115,0.360450,0.887939,0.345021,1.274465,0.741954,0.460649,ENSG00000282936,protein_coding,CTACTGGAACGCCCCCCTCAATCTAGCCTCCCCCACATAACTCTCT...,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",0,0,2


In [51]:
sum_vec = data_org.iloc[:, :9].sum(axis=1)
data2 = data_org.iloc[:, :9].divide(sum_vec, axis='index')
data = pd.concat([data2, data_org['seq']], axis=1)
data

,ERM,KDEL,LMA,MITO,NES,NIK,NLS,NUCP,OMM,seq
0,0.207338,0.128871,0.079991,0.044906,0.082833,0.088109,0.061681,0.106670,0.199601,ACCTTGTATTAGGTATTTATTTCCACAAAAGTTTGATGCTTACAAC...
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.922605,0.077395,TGTGCACAGAAGTTATATACATATATGGGTATATCTATGTAACAAA...
2,0.064346,0.125935,0.108187,0.082002,0.091019,0.160823,0.142649,0.131566,0.093474,TACTTTATGCAAAAAAAAATATACATTTATTTATAGGTCTCAATAC...
3,0.141112,0.094932,0.211394,0.028049,0.102672,0.102581,0.124625,0.090767,0.103868,TGACTTTCAAACCATTTTAATATTTCAAATATTCCAGAACAATCCC...
4,0.172315,0.115240,0.132014,0.039526,0.115743,0.088187,0.142063,0.105034,0.089878,AACCCGCTCGGGTCCCCTTCCACACTGTGGAAGCTTTGTTCTTTCG...
...,...,...,...,...,...,...,...,...,...,...
13805,0.000000,0.000000,0.734451,0.109981,0.073566,0.000000,0.082002,0.000000,0.000000,GGGAAGAAAGGAGCCTGACTCTTATGATGGAATAACCACAAATCAG...
13806,0.067811,0.056029,0.110082,0.010354,0.050838,0.430809,0.110393,0.000000,0.163685,GTGTCGGACGGCATGACAGGCAGCAATCCTGTGTCCCCTGCCTCAT...
13807,0.006797,0.081510,0.077941,0.046627,0.059330,0.330415,0.263028,0.045733,0.088620,ATGGCGGGGACCTCCGCGCCAGGCAGCAAGAGGCGGAGCGAGCCCC...
13808,0.000000,0.077421,0.315861,0.053726,0.132350,0.051426,0.189963,0.110591,0.068661,CTACTGGAACGCCCCCCTCAATCTAGCCTCCCCCACATAACTCTCT...


In [52]:
# One hot encode the 'seq' attribute of the above table
mapping = { 
            'A': 0,
            'C': 1,
            'G': 2,
            'T': 3
            }
one_hot_encode_lam = lambda seq: to_categorical([mapping[x] for x in seq])
data_one_hot_karas = data['seq'].apply(one_hot_encode_lam)

data_one_hot_karas

# Now just injecting this modified 'seq' back into the pandas frame 
data_new_karas =  pd.concat([data.iloc[:, :9], data_one_hot_karas], axis=1)

data_new_karas

,ERM,KDEL,LMA,MITO,NES,NIK,NLS,NUCP,OMM,seq
0,0.207338,0.128871,0.079991,0.044906,0.082833,0.088109,0.061681,0.106670,0.199601,"[[1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [..."
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.922605,0.077395,"[[0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 1.0, 0.0], [..."
2,0.064346,0.125935,0.108187,0.082002,0.091019,0.160823,0.142649,0.131566,0.093474,"[[0.0, 0.0, 0.0, 1.0], [1.0, 0.0, 0.0, 0.0], [..."
3,0.141112,0.094932,0.211394,0.028049,0.102672,0.102581,0.124625,0.090767,0.103868,"[[0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 1.0, 0.0], [..."
4,0.172315,0.115240,0.132014,0.039526,0.115743,0.088187,0.142063,0.105034,0.089878,"[[1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [..."
...,...,...,...,...,...,...,...,...,...,...
13805,0.000000,0.000000,0.734451,0.109981,0.073566,0.000000,0.082002,0.000000,0.000000,"[[0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 1.0, 0.0], [..."
13806,0.067811,0.056029,0.110082,0.010354,0.050838,0.430809,0.110393,0.000000,0.163685,"[[0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 0.0, 1.0], [..."
13807,0.006797,0.081510,0.077941,0.046627,0.059330,0.330415,0.263028,0.045733,0.088620,"[[1.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0], [..."
13808,0.000000,0.077421,0.315861,0.053726,0.132350,0.051426,0.189963,0.110591,0.068661,"[[0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0], [..."


In [81]:
# Splitting for 5fold

folds_total = 5

lengths = data_new_karas['seq'].apply(lambda x: len(x))

# max_length = lengths.max()

max_length = 2000

print(max_length)

def pad(x):
    if len(x) < max_length:
        return np.concatenate((x, np.array([np.array([0.0, 0.0, 0.0, 0.0]) for i in range(max_length - len(x))])), axis=0)
    else:
        return x[:max_length]

X = data_new_karas['seq'].apply(pad)
# X = data_new_karas['seq']

# y = pd.DataFrame(columns=['Labels'])

# X = tf.data.Dataset.from_tensor_slices(dict(X))

y = data_new_karas.iloc[:, :9].values.tolist()
# y = y.apply(lambda x: np.array(x))

# y = tf.data.Dataset.from_tensor_slices(dict(y))

X = np.stack(X).transpose((0, 2, 1))
y = np.stack(y)

# X = tf.convert_to_tensor(X)
# y = tf.convert_to_tensor(y)

# kf = KFold(n_splits=folds_total, shuffle=True, random_state=1234)

# folds = kf.split(X, y)

kf = KFold(n_splits=folds_total, shuffle=True, random_state=1)

folds = kf.split(X,y)

print(X.shape)
print(y.shape)
# folds now contains a list of lists. Each sublist contains all the indices for the pandas data entries to be used in the respective fold

2000
(13810, 4, 2000)
(13810, 9)


In [ ]:
# TODO: what is this?

template = [0] * 24  # dim([a,c,g,t]) * dim([f,t,i,h,m,s])
combined_encoding = OrderedDict()
combined_encoding['UNK'] = template
for i, (key_seq, key_ann) in enumerate(
        itertools.product(['A', 'C', 'G', 'T', 'N'], ['F', 'T', 'I', 'H', 'M', 'S'])):
    tmp = template.copy()
    if key_seq == 'N':
        for n in ['A', 'C', 'G', 'T']:
            tmp[np.nonzero(combined_encoding[n + key_ann])[0][0]] = 0.25
        combined_encoding[key_seq + key_ann] = tmp
    else:
        tmp[i] = 1  # normal one-hot encoding as it is...
        combined_encoding[key_seq + key_ann] = tmp
encoding_keys = list(combined_encoding.keys())
encoding_vectors = np.array(list(combined_encoding.values()))
encoding_vectors

### **2** |  Importing models...

We use as baseline model the [RNATracker](https://github.com/HarveyYan/RNATracker/blob/master/Models/cnn_bilstm_attention.py) model and a CNN model.


In [82]:
# Import NN
# from RNAtracker import RNATracker
from cnn_model import CNN_MLRG

# TODO: do a normal CNN
import keras
from keras.layers import Conv1D, Dense, Flatten, Input, MaxPooling1D, SpatialDropout1D

# Set paths for model output
try:
    os.mkdir('model_outputs/')
except Exception as e:
    print("Output folder already exists")

model_output_folder = 'model_outputs/'

model = keras.Sequential()

model.add(Conv1D(32, kernel_size=3, activation='relu', input_shape=(4, 2000)))
# model.add(Conv1D(32, kernel_size=3, activation='relu'))
# model.add(Conv1D(64, kernel_size=3, activation='relu'))
model.add(SpatialDropout1D(rate=0.2))
model.add(MaxPooling1D(pool_size=2))

model.add(Dense(9, activation="softmax"))

model.add(Flatten())

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=['accuracy'])

model.summary()

Output folder already exists
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 2, 32)             192032    
                                                                 
 spatial_dropout1d (SpatialD  (None, 2, 32)            0         
 ropout1D)                                                       
                                                                 
 max_pooling1d (MaxPooling1D  (None, 1, 32)            0         
 )                                                               
                                                                 
 dense (Dense)               (None, 1, 9)              297       
                                                                 
 flatten (Flatten)           (None, 9)                 0         
                                                                 
Total params: 192,329
Train

In [83]:

# TODO: with understand what we have to predict, we can allocate X and y
# Also: the kwargsvalues are hyperparameters of which we will select default values from the RNAtracker repo

VALIDATION_ACCURACY = []
VALIDATION_LOSS = []

print("Start training...")

for i, (train_indices, test_indices) in enumerate(folds):
    print('Evaluating KFolds {}/{}'.format(str(i + 1), str(folds_total)))
    # model = RNATracker(max_len, nb_classes, model_output_folder, kfold_index=i)
    #model.build_model(nb_filters=kwargs['nb_filters'], filters_length=kwargs['filters_length'],
    #                          pooling_size=kwargs['pooling_size'], lstm_units=kwargs['lstm_units'],
    #                          embedding_vec=encoding_vectors)

    # model.build_model_advanced_masking(nb_filters=kwargs['nb_filters'],
     #                                              filters_length=kwargs['filters_length'],
      #                                             pooling_size=kwargs['pooling_size'],
       #                                            lstm_units=kwargs['lstm_units'],
        #                                           embedding_vec=encoding_vectors)
    

    # model.train(X[train_indices], y[train_indices], batch_size, kwargs['epochs'])

    # model.evaluate(X[test_indices], y[test_indices], dataset)

    # K.clear_session()
    model = model

    checkpoint = tf.keras.callbacks.ModelCheckpoint(model_output_folder + f"cnn{i}.h5",
							monitor='val_accuracy', verbose=1,
							save_best_only=True, mode='max')

    callbacks_list = [checkpoint]

    history = model.fit(X[train_indices], y[train_indices], epochs=3,
                        callbacks=callbacks_list, validation_data=(X[test_indices], y[test_indices]))

    model.load_weights(f"model_outputs/cnn{i}.h5")

    results = model.evaluate(X[test_indices])
    results = dict(zip(model.metrics_names,results))

    VALIDATION_ACCURACY.append(results['accuracy'])
    VALIDATION_LOSS.append(results['loss'])

    tf.keras.backend.clear_session()

print("Training done!")

Start training...
Evaluating KFolds 1/5
Epoch 1/3
335/346 [============================>.] - ETA: 0s - loss: 2.1832 - accuracy: 0.1542
Epoch 1: val_accuracy improved from -inf to 0.15206, saving model to model_outputs\cnn0.h5
346/346 [==============================] - 1s 2ms/step - loss: 2.1830 - accuracy: 0.1536 - val_loss: 2.1745 - val_accuracy: 0.1521
Epoch 2/3
333/346 [===========================>..] - ETA: 0s - loss: 2.1734 - accuracy: 0.1546
Epoch 2: val_accuracy did not improve from 0.15206
346/346 [==============================] - 1s 2ms/step - loss: 2.1733 - accuracy: 0.1552 - val_loss: 2.1715 - val_accuracy: 0.1517
Epoch 3/3
333/346 [===========================>..] - ETA: 0s - loss: 2.1706 - accuracy: 0.1537
Epoch 3: val_accuracy improved from 0.15206 to 0.15243, saving model to model_outputs\cnn0.h5
87/87 [==============================] - 0s 477us/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
Evaluating KFolds 2/5
Epoch 1/3
328/346 [===========================>..] - ETA: 